In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix
import sys 
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
from VisUtils import *
import pandas as pd
import numpy as np

In [ ]:
result_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/'
datapath1 = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK2/bind/'
pickle_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_pickles/'
# testX = pd.read_csv(datapath1+'NEK2_binding_moe_UNDER_testX.csv',header=None).to_numpy()
# testy = pd.read_csv(datapath1+'NEK2_binding_moe_UNDER_test_y.csv',header=None).to_numpy()
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition'] 
nek_list = ["2", "3", "5", "9"]
nektype = ['binding','inhibition']
feat_types = ['moe', 'mfp']
samplings = ['scaled', 'UNDER' , 'SMOTE', 'ADASYN']
model_types = ['RF','RF_BCW', 'BRFC', 'BRFC_BCW']
# with open(pickle_path+'NEK2_binding_moe_UNDER_BRFC_BCW.pkl', 'rb') as f: 
#     model = pickle.load(f)
# with open(pickle_path+'NEK2_binding_moe_UNDER_RF_BCW_GS.pkl', 'rb') as f: 
#     bestmodel = pickle.load(f)

In [ ]:
def df_tree_probs(model, x_data): 
    test_proba0_df = pd.DataFrame()
    test_proba1_df = pd.DataFrame()
    for tree_num in np.arange(len(model.estimators_)):
        test_proba_temp = model.estimators_[tree_num].predict_proba(x_data)
        test_proba0_df[tree_num] = test_proba_temp[:,0]
        test_proba1_df[tree_num] = test_proba_temp[:,1]
    std0 = test_proba0_df.std(axis=1)
    std1 = test_proba1_df.std(axis=1)
    return std0, std1

In [ ]:
for n in nek_list:
    for i in nektype: 
        for feat in feat_types: 
            for samp in samplings:  
                for rf in model_types:  # RF, RF_BCW, BRFC, BRFC_BCW
                    if i == 'binding':
                        datapath = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/bind/'
                        testX = pd.read_csv(f'{datapath}NEK{n}_binding_{feat}_{samp}_testX.csv',header=None).to_numpy()
                        # testy=pd.read_csv(f'{datapath}NEK{n}_binding_{feat}_{samp}_test_y.csv',header=None).to_numpy()
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        df = pd.read_csv(f'{result_path}{model_name}_test.csv')
                        
                        
                        predictions = df['prediction'].to_numpy()
                        test_y =  df['y'].to_numpy()
                        with open(pickle_path+model_name+'.pkl', 'rb') as f: 
                            model = pickle.load(f)
                            std0, std1 = df_tree_probs(model, testX)
                            prob_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs/'
                            # test_proba0_df.to_csv(prob_path+model_name+'_proba0.csv',index=False)
                            # test_proba1_df.to_csv(prob_path+model_name+'_proba1.csv',index=False)
                            swarm_stdprob_RF(predictions, testX, test_y,std0, std1, title=f'{model_name}')
                            print(type(std0))
                    if n in ['2', '9'] and i == 'inhibition': 
                        datapath = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/inhib/'
                        testX = pd.read_csv(f'{datapath}NEK{n}_inhibition_{feat}_{samp}_testX.csv',header=None).to_numpy()
    
                        model_name = f'NEK{n}_{i}_{feat}_{samp}_{rf}'
                        df = pd.read_csv(f'{result_path}{model_name}_test.csv')
                        
                        
                        predictions = df['prediction'].to_numpy()
                        test_y =  df['y'].to_numpy()
                        with open(pickle_path+model_name+'.pkl', 'rb') as f: 
                            model = pickle.load(f)
                            std0, std1 = df_tree_probs(model, testX)
                            prob_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs/'
                            
                            swarm_stdprob_RF(predictions, testX, test_y,std0, std1, title=f'{model_name}')
                        
                        

In [ ]:
for n in ['2', '3','5','9']:
    for samp in samplings: 
        combined_feats= []
        df = pd.DataFrame()
        std0s = pd.DataFrame() 
        std1s = pd.DataFrame() 
        for rf in model_types:  # RF, RF_BCW, BRFC, BRFC_BCW
            for feat in feat_types: 
                NEK = f'NEK{n}_binding'
                datapath = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/bind/'
                testX = pd.read_csv(f'{datapath}NEK{n}_binding_{feat}_{samp}_testX.csv',header=None).to_numpy()
  
                model_name = f'NEK{n}_binding_{feat}_{samp}_{rf}'
                df = pd.read_csv(f'{result_path}{model_name}_test.csv')
                
                predictions = df['prediction'].to_numpy()
                test_y =  df['y'].to_numpy()
                with open(pickle_path+model_name+'.pkl', 'rb') as f: 
                    model = pickle.load(f)
                    std0, std1 = df_tree_probs(model, testX)
                    prob_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs/'
                    
                    this_std0 = pd.DataFrame(std0) 
                    this_std1 = pd.DataFrame(std1) 
                    label0 = f'std0_{feat}'
                    label1 = f'std1_{feat}'
                    std0s[label0] = this_std0 
                    std1s[label1] = this_std1 

            
            print('feat: '+feat)
            df = pd.concat([std0s, std1s],axis=1)
            df['model'] = model_name 
            df['NEK'] = NEK
            if samp == 'scaled': 
                
                df['strategy'] = 'raw'
            else: 
                df['strategy'] = samp
            
            df['feat_type'] = feat 
            df['RF_type'] = rf 
            display(df)
            print('done')
            df.to_csv(f'{prob_path}{NEK}_std_prob_{samp}_{rf}.csv')
                


In [ ]:
for n in ['2', '9']:
    for samp in samplings: 
        combined_feats= []
        df = pd.DataFrame()
        std0s = pd.DataFrame() 
        std1s = pd.DataFrame() 
        for rf in model_types:  # RF, RF_BCW, BRFC, BRFC_BCW
            for feat in feat_types: 
                NEK = f'NEK{n}_inhibition'
                datapath = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{n}/inhib/'
                testX = pd.read_csv(f'{datapath}NEK{n}_inhibition_{feat}_{samp}_testX.csv',header=None).to_numpy()
                # testy=pd.read_csv(f'{datapath}NEK{n}_binding_{feat}_{samp}_test_y.csv',header=None).to_numpy()
                model_name = f'NEK{n}_inhibition_{feat}_{samp}_{rf}'
                df = pd.read_csv(f'{result_path}{model_name}_test.csv')
                
                predictions = df['prediction'].to_numpy()
                test_y =  df['y'].to_numpy()
                with open(pickle_path+model_name+'.pkl', 'rb') as f: 
                    model = pickle.load(f)
                    std0, std1 = df_tree_probs(model, testX)
                    prob_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs/'
               
                    this_std0 = pd.DataFrame(std0) 
                    this_std1 = pd.DataFrame(std1) 
                    label0 = f'std0_{feat}'
                    label1 = f'std1_{feat}'
                    std0s[label0] = this_std0 
                    std1s[label1] = this_std1 

            
            print('feat: '+feat)
            df = pd.concat([std0s, std1s],axis=1)
            df['model'] = model_name 
            df['NEK'] = NEK
            if samp == 'scaled': 
                
                df['strategy'] = 'raw'
            else: 
                df['strategy'] = samp
            
            df['feat_type'] = feat 
            df['RF_type'] = rf 
            display(df)
            print('done')
            df.to_csv(f'{prob_path}{NEK}_std_prob_{samp}_{rf}.csv',index=False)
   

# Plotting one mfp,moe at a time for each RF type and sampling strategy

In [ ]:
prob_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_results/proba_dfs/'
neks = ['NEK2_binding', 'NEK2_inhibition','NEK3_binding', 'NEK5_binding', 'NEK9_binding','NEK9_inhibition']
figure_path = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/UQ_RF/figures/RF_std_prob_swarm/'
for nek in neks: 
    for samp in samplings: 
        for rf in model_types:
            
            moe_pred = pd.read_csv(f'{result_path}{nek}_moe_{samp}_{rf}_test.csv')
            mfp_pred = pd.read_csv(f'{result_path}{nek}_mfp_{samp}_{rf}_test.csv')
            stdprobs = pd.read_csv(f'{prob_path}{nek}_std_prob_{samp}_{rf}.csv')
            std0_moe = stdprobs['std0_moe']
            std1_moe = stdprobs['std1_moe']
            std0_mfp = stdprobs['std0_mfp']
            std1_mfp  = stdprobs['std1_mfp']
            moe_df = get_swarm_data(moe_pred['prediction'].to_numpy(), moe_pred['y'].to_numpy(), std1_moe.to_numpy(), std0_moe.to_numpy(), 'moe')
            mfp_df = get_swarm_data(mfp_pred['prediction'].to_numpy(), mfp_pred['y'].to_numpy(), std1_mfp.to_numpy(), std0_mfp.to_numpy(), 'mfp')
            df = pd.concat([moe_df, mfp_df])
            if (samp == 'scaled'): 
                samp = 'raw' 
            title = f'{nek} {samp} {rf} std prob'
            plot_swarm_std_prob_RF(df, figure_path, title, f'{nek}_{samp}_{rf}_std_prob')
            if (samp == 'raw'): 
                samp = 'scaled' 
            
            